In [1]:
import torch
import torchvision.transforms as tvt
from torch.utils.data import ConcatDataset, DataLoader
from torchmetrics import AUROC  # additional dependency
from torchvision.datasets import CIFAR100
from tqdm.notebook import tqdm

from oodtk import NegativeEnergy, Softmax, ODIN, MCD, Mahalanobis
from oodtk.dataset.img import Textures, CIFAR10C, CIFAR10P, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
from oodtk.model import WideResNet
from oodtk.utils import is_unknown, OODMetrics
from oodtk.transforms import ToRGB


In [2]:
torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([ToRGB(), tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

# setup data
dataset_train = CIFAR100(root="data", train=True, download=True, transform=trans)
dataset_in_test = CIFAR100(root="data", train=False, transform=trans)
dataset_out_test1 = Textures(root="data", download=True, transform=trans)
dataset_out_test2 = LSUNCrop(root="data", download=True, transform=trans)
dataset_out_test3 = LSUNResize(root="data", download=True, transform=trans)
dataset_out_test4 = TinyImageNetResize(root="data", download=True, transform=trans)
dataset_out_test5 = TinyImageNetCrop(root="data", download=True, transform=trans)
dataset_test = dataset_in_test + dataset_out_test1 + dataset_out_test2 + dataset_out_test3 + dataset_out_test4 + dataset_out_test5
train_loader = DataLoader(dataset_train, batch_size=128, num_workers=20)
test_loader = DataLoader(dataset_test, batch_size=128, num_workers=20)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
model = WideResNet.from_pretrained("oe-cifar100-tune", num_classes=100).eval().cuda()
method = Softmax(model).cuda()
metrics = OODMetrics()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        metrics.update(method.predict(x), y)

print(metrics.compute())
metrics.reset()

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to a

{'AUROC': 0.8719539046287537, 'AUPR-IN': 0.96767258644104, 'AUPR-OUT': 0.636948823928833, 'ACC95TPR': 0.8594536185264587, 'FPR95TPR': 0.5537999868392944}


In [4]:
model = WideResNet.from_pretrained("er-cifar100-tune", num_classes=100).eval().cuda()
energy = NegativeEnergy(model)
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()
        metrics.update(energy.predict(x), y)

print(metrics.compute())
metrics.reset()

Downloading: "https://github.com/wetliu/energy_ood/raw/master/CIFAR/snapshots/energy_ft/cifar100_wrn_s1_energy_ft_epoch_9.pt" to /home/ki/.cache/torch/hub/checkpoints/cifar100_wrn_s1_energy_ft_epoch_9.pt


  0%|          | 0.00/8.70M [00:00<?, ?B/s]

  0%|          | 0/435 [00:00<?, ?it/s]

{'AUROC': 0.8667274117469788, 'AUPR-IN': 0.9668105244636536, 'AUPR-OUT': 0.581452488899231, 'ACC95TPR': 0.847304105758667, 'FPR95TPR': 0.621399998664856}


In [5]:
model = WideResNet.from_pretrained("cifar100-pt", num_classes=100).eval().cuda()
method = Softmax(model)
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        metrics.update(method.predict(x), y)

print(metrics.compute())
metrics.reset()

Downloading: "https://github.com/wetliu/energy_ood/raw/master/CIFAR/snapshots/pretrained/cifar100_wrn_pretrained_epoch_99.pt" to /home/ki/.cache/torch/hub/checkpoints/cifar100_wrn_pretrained_epoch_99.pt


  0%|          | 0.00/8.66M [00:00<?, ?B/s]

  0%|          | 0/435 [00:00<?, ?it/s]

{'AUROC': 0.7962963581085205, 'AUPR-IN': 0.9377251267433167, 'AUPR-OUT': 0.5602498054504395, 'ACC95TPR': 0.8529654741287231, 'FPR95TPR': 0.589900016784668}


In [6]:
model = WideResNet.from_pretrained("cifar100-pt", num_classes=100).eval().cuda()
method = NegativeEnergy(model)
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        metrics.update(method.predict(x), y)

print(metrics.compute())
metrics.reset()

  0%|          | 0/435 [00:00<?, ?it/s]

{'AUROC': 0.8572402596473694, 'AUPR-IN': 0.961593747138977, 'AUPR-OUT': 0.6134926080703735, 'ACC95TPR': 0.8599209189414978, 'FPR95TPR': 0.5511999726295471}


In [7]:
model = WideResNet.from_pretrained("cifar100-pt", num_classes=100).eval().cuda()
method = ODIN(model, eps=0.002, norm_std=std)
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        metrics.update(method.predict(x), y)

print(metrics.compute())
metrics.reset()

  0%|          | 0/435 [00:00<?, ?it/s]

{'AUROC': 0.8241766691207886, 'AUPR-IN': 0.9468352794647217, 'AUPR-OUT': 0.5941141843795776, 'ACC95TPR': 0.8576563596725464, 'FPR95TPR': 0.5637999773025513}


In [8]:
model = WideResNet.from_pretrained("cifar100-pt", num_classes=100).eval().cuda()
method = MCD(model)
metrics = OODMetrics()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        metrics.update(method.predict(x), y)

print(metrics.compute())
metrics.reset()

  0%|          | 0/435 [00:00<?, ?it/s]

{'AUROC': 0.8310259580612183, 'AUPR-IN': 0.9378737211227417, 'AUPR-OUT': 0.6571603417396545, 'ACC95TPR': 0.8702372312545776, 'FPR95TPR': 0.49380001425743103}
